# 나무 위키 RAG를 위한 메타 데이터를 활용한 벡터 스토어 생성 데모 노트북

### FaissMetaVectorStore
- 가장 많이 쓰이는벡터 스토어인 `FAISS`를 상속받아 활용한 벡터스토어
- 기존 벡터데이터가 오로지 `Document`들의 `page_content`만 임베딩하는 것과 달리
- 사용자가 벡터스토어 생성시 `Document`들이 공통으로 가지고 있는 `metadata`들의 필드 중 특정 필드를 페이지 컨텐츠와 독립적으로 임베딩하여 저장함
    - `FAISS`가 `index`라는 벡터 저장소를 하나만 가지고 있는 것과 달리 `FaissMetaVectorStore`는 `index`, `metaindex` 두개를 가지고 있음
    - `index` : `page_content`를 임베딩한 것
    - `metaindex`: `metadata` 중 특정 key값들의 value들을 concat한 문자열을 임베딩해 갖고 있음
- 사용법 예시: `vs_meta = FaissMetaVectorStore.from_documents(docs, embedding = EMBEDDING_FUNC, metadata_fields= ["abs_page_toc_item", "base_page_url"])` => abs_page_toc_item와 base_page_url이 metadata에 공통으로 있는 key 들이어야 함

- as_retriever()로 리트리버를 생성 시 두 좋류의 리트리버를 생성 가능
    - `vs.as_retriever(vectorStoreType='page_content', search_kwargs={'k': k})`: 일반적인 페이지 컨텐츠 임베딩에 따른 리트리버
    - `vs.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})`: 위에서 설정한 메타 데이터 임베딩에 따른 리트리버
    - 두 리트리버를 `EnsembleRetriever`로 섞어 사용 가능 하다.


## 모델 테스트 준비

> 관련 패키지 import

In [2]:
import os
from operator import itemgetter
from time import time

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import ConfigurableField

from namu_loader import NamuLoader

from langchain_community.vectorstores.faiss import FAISS
from FaissMetaVectorStore import FaissMetaVectorStore # Custom 제작 메타 데이터 벡터스토어

> 경로, API 키 설정 등

In [10]:
cur_notebook_dir = os.getcwd()
os.chdir("../")
base_dir = os.getcwd()
key_dir = os.path.join(base_dir, 'keys')
os.chdir(cur_notebook_dir)
print(os.getcwd())

from dotenv import load_dotenv
print(load_dotenv(dotenv_path= os.path.join(key_dir, ".env")))
os.chdir(cur_notebook_dir)

EMBEDDING_FUNC = OpenAIEmbeddings()

c:\Users\thdwo\Documents\Github\proj_artist_info_gen
True


> 기타 체인, 답변 관련 함수 정의, LLM 모델 정의

In [16]:
def get_documnet_from_namuwiki(url, NamuLoader_obj, hop = 1):
    loader = NamuLoader_obj(url, hop, verbose = True)
    docs = loader.load()

    # splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=500,
    #     chunk_overlap=20
    #     )
    # docs = splitter.split_documents(docs)
    return docs

def get_chain(retriever, model):
    prompt = ChatPromptTemplate.from_template("""
        Answer the users's QUESTION based on the CONTEXT documents. Use the language that the user types in.
        If you can not find an appropriate answer, return "No Answer found.".
        CONTEXT: {context}
        QUESTION: {question}
    """)

    def concat_docs(docs) -> str: # retriever가 반환한 모든 Document들의 page_content를 하나의 단일 string으로 붙여주는 함수
        return "".join(doc.page_content for doc in docs)

    chain = {
        "context" : itemgetter('question') | retriever | concat_docs
        , "question" : itemgetter('question') | RunnablePassthrough()
    } | prompt| model | StrOutputParser()

    return chain

def get_answer(chain, question,  temp = .1, max_tokens = 2048):
    for model_name in ['claude3_haiku', 'claude3_5_sonnet', 'gpt3_5', 'gpt4o']:
        try:
            start = time()
            result = chain.with_config(configurable={
                        "llm": model_name, 
                        "temparature": temp,
                        "max_tokens": max_tokens
                    }).invoke({"question": question})
            print("\n\nModel: {} \t\t elapsed_time: {} \n\n Answer: {}".format(model_name, round(time() - start, 1), result))
            print("\n====================================================================================================")
        except:
            print("\n\nModel: {} \t\t Answer: {}".format(model_name, "Error."))
            print("\n====================================================================================================")

>  LLM 모델 정의, 나무위키 문서 데이터 Loading

In [76]:
llm = (
    ChatAnthropic(model_name='claude-3-5-sonnet-20240620')
    .configurable_alternatives(
        ConfigurableField(
            id = 'llm',
            name="LLM Model",
            description="The base LLM model",
        ),
        default_key="claude3_5_sonnet",
        claude3_haiku=ChatAnthropic(model_name='claude-3-haiku-20240307'),
        gpt4o = ChatOpenAI(model = 'gpt-4o'),
        gpt3_5 = ChatOpenAI(model = 'gpt-3.5-turbo'),
    )
    .configurable_fields(
        temperature=ConfigurableField(
            id="temperature",
            name="LLM Temperature",
            description="The temperature of the LLM",
        ),
        max_tokens = ConfigurableField(
            id="max_token",
            name="Maximum input Tokens",
            description="Maximum limit of input Tokens",
        ),
    )
)

url = 'https://namu.wiki/w/ILLIT'
docs = get_documnet_from_namuwiki(url = url , NamuLoader_obj = NamuLoader, hop = 1)


Document of ILLIT


================== Table of Contents  ==================

0. PROFILE
1. 개요
2. 멤버
	2.1. 멤버 간 케미
	2.2. 알유넥스트에서의 멤버들
	2.3. 알유넥스트의 멤버별 평가 내역
3. 특징
	3.1. 로고
	3.2. 그룹명
	3.3. 콘셉트
	3.4. 실력
		3.4.1. 보컬
		3.4.2. 댄스
		3.4.3. 랩
	3.5. 비주얼
	3.6. 인기 및 기록
	3.7. 세계관
4. 음반
5. 뮤직비디오
6. 활동
	6.1. 음악 방송 직캠
	6.2. 콘텐츠
		6.2.1. 숏폼
		6.2.2. Weverse
	6.3. 공연 및 행사
	6.4. 광고 및 화보
7. 수상 경력
8. 빌보드
	8.1. 빌보드 Hot 100
	8.2. 빌보드 200
	8.3. 빌보드 글로벌 200
	8.4. 빌보드 글로벌 200 (미국 제외)
9. 팬덤
	9.1. 응원법
	9.2. 굿즈
10. 여담
11. 논란 및 사건 사고
12. 역대 프로필 사진
EOD. FOOTNOTES
Main Doc ILLIT                                              Start (hop : 0/1)
  Sub Doc ILLIT/멤버 간 케미                            Start (hop : 1/1)
  Sub Doc ILLIT/멤버 간 케미                            Done! (hop : 1/1, elapsed_time: 1.1 seconds)
  Sub Doc SUMMER MOON                              Start (hop : 1/1)
  Sub Doc SUMMER MOON                              Done! (hop : 1/1, elapsed_time: 0.6 seconds)
  Sub Doc ILLIT/음반                                

## 벡터 스토어 비교

In [77]:
k = 5 ## 반환할 최대 문서 개수

question1 = "음반에 대해 설명해줘"


### Vanilla FAISS 벡터 스토어

In [78]:
vs = FAISS.from_documents(docs, embedding = EMBEDDING_FUNC)
ret = vs.as_retriever(search_kwargs={'k': k})
chain = get_chain(ret, llm)
get_answer(chain, question1)



Model: claude3_haiku 		 elapsed_time: 2.5 

 Answer: ILLIT의 데뷔 앨범에 대한 정보는 제가 찾은 자료에서 자세히 언급되어 있지 않습니다. 하지만 ILLIT의 멤버 간 케미, 공연 및 행사 등 전반적인 활동 내용은 CONTEXT 문서들에서 다루고 있습니다. 데뷔 앨범에 대한 추가 정보는 https://namu.wiki/w/%EB%AF%BC%ED%9D%AC%EC%A7%84-HYBE%20%EA%B0%84%20ADOR%20%EA%B2%BD%EC%98%81%EA%B6%8C%20%EB%B6%84%EC%9F%81 문서에서 찾아볼 수 있을 것 같습니다.



Model: claude3_5_sonnet 		 elapsed_time: 4.6 

 Answer: 죄송합니다. 주어진 문맥에서 ILLIT의 음반에 대한 구체적인 설명을 찾을 수 없습니다. 단, "데뷔 앨범 수록곡 중 My World에 등장한 바가 있다"라는 언급이 있지만, 이것만으로는 음반 전체에 대한 설명을 제공하기에 충분하지 않습니다. 더 자세한 정보가 필요하다면 추가적인 자료를 참조해야 할 것 같습니다.



Model: gpt3_5 		 elapsed_time: 2.4 

 Answer: 다음 문서를 참조하시면 음반에 대한 자세한 설명을 확인하실 수 있습니다: https://namu.wiki/w/%EB%AF%BC%ED%9D%AC%EC%A7%84-HYBE%20%EA%B0%84%20ADOR%20%EA%B2%BD%EC%98%81%EA%B6%8C%20%EB%B6%84%EC%9F%81



Model: gpt4o 		 elapsed_time: 0.9 

 Answer: No Answer found.



##### 답변에 사용된 Document 보기

In [79]:
ret.get_relevant_documents(query="앨범")

[Document(page_content='데뷔 앨범 수록곡 중 My World에 등장한 바가 있다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/SUMMER%20MOON', 'page_hop': 1, 'parent_page_index': '3.7.', 'parent_page_toc_item': '세계관', 'abs_page_toc_item': '특징/세계관//여담', 'index': '2.', 'toc_item': '여담', 'ancestor_toc_item': ''}),
 Document(page_content='위버스 샵 아일릿 페이지(글로벌)', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EA%B5%BF%EC%A6%88', 'page_hop': 1, 'parent_page_index': '9.2.', 'parent_page_toc_item': '굿즈', 'abs_page_toc_item': '팬덤/굿즈//위버스샵', 'index': '2.', 'toc_item': '위버스샵', 'ancestor_toc_item': ''}),
 Document(page_content='ILLIT의 공연 및 행사를 서술한 문서이다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EA%B3%B5%EC%97%B0

### MetaFAISS 벡터 스토어
- 새롭게 정의한 FaissMetaVectorStore를 통해 생성한 메타 벡터스토어 리트리버를 활용한 결과
- 메타 데이터 중 `abs_page_toc_item`와 `base_page_url`만을 사용해 임베딩한 뒤 벡터스토어에 저장
- 입력 쿼리와 위의 `abs_page_toc_item`와 `base_page_url`만 비교해 CONTEXT Document들을 반환, 답변에 사용

In [80]:
vs_meta = FaissMetaVectorStore.from_documents(docs, embedding = EMBEDDING_FUNC, metadata_fields= ["abs_page_toc_item", "base_page_url"])
ret_meta = vs_meta.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})
chain_meta = get_chain(ret_meta, llm)
get_answer(chain_meta, question1)



Model: claude3_haiku 		 elapsed_time: 2.8 

 Answer: ILLIT이 발매한 음반에 대해 자세하게 설명하겠습니다:

SUPER REAL ME (미니 1집)
- 2024년 3월 25일 발매
- ILLIT의 첫 번째 미니 앨범

SUPER REAL ME (Sped Up) (리믹스 EP)
- 2024년 4월 1일 발매
- SUPER REAL ME 타이틀곡의 스피드업 리믹스 버전을 수록

Magnetic (Remixes) (리믹스 싱글)
- 2024년 4월 19일 발매
- Magnetic 곡의 리믹스 버전을 수록

음반에 대한 자세한 설명은 위와 같습니다. 추가로 알고 싶은 사항이 있으시면 더 자세히 설명 드리겠습니다.



Model: claude3_5_sonnet 		 elapsed_time: 5.6 

 Answer: ILLIT이 발매한 음반에 대해 다음과 같이 설명할 수 있습니다:

1. "SUPER REAL ME": 2024년 3월 25일에 발매된 미니 1집 앨범입니다.

2. "SUPER REAL ME (Sped Up)": 2024년 4월 1일에 발매된 리믹스 EP입니다.

3. "Magnetic (Remixes)": 2024년 4월 19일에 발매된 리믹스 싱글입니다.

이 세 음반은 ILLIT의 초기 활동을 보여주는 작품들로, 정규 앨범과 리믹스 버전을 포함하고 있습니다. 특히 "SUPER REAL ME"는 그들의 첫 번째 미니 앨범으로, 그룹의 음악적 정체성을 처음으로 선보인 작품이라고 볼 수 있습니다.



Model: gpt3_5 		 elapsed_time: 2.9 

 Answer: 걸그룹 ILLIT이 발매한 음반은 SUPER REAL ME 미니 1집이 2024년 3월 25일에 발매되었고, SUPER REAL ME (Sped Up) 리믹스 EP는 2024년 4월 1일에 발매되었으며, Magnetic (Remixes) 리믹스 싱글은 2024년 4월 19일에 발매되었습니다.



Model: gpt4o 		 

##### 답변에 사용된 Document 보기

In [82]:
ret_meta.get_relevant_documents('음반')

[Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//국내 음반', 'index': '2.', 'toc_item': '국내 음반', 'ancestor_toc_item': ''}),
 Document(page_content='걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. ,,,,SUPER REAL ME미니 1집2024. 03. 25.,SUPER REAL ME (Sped Up)리믹스 EP2024. 04. 01.,Magnetic (Remixes)리믹스 싱글2024. 04. 19.,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https:

#### 두번째 질문

#### Vanilla FAISS 벡터 스토어

In [99]:
question2 = '연습생 생활이 얼마나 길었어?'
get_answer(chain, question2)



Model: claude3_haiku 		 elapsed_time: 2.6 

 Answer: 문서에 따르면 ILLIT의 멤버들인 이로하, 모카, 윤아, 민주는 전부 상당한 기간 동안 연습생 생활을 했던 것으로 나타나고 있습니다. 특히 이로하는 만 3세부터 전문적인 춤 연습을 해왔을 정도로 오랜 연습생 생활을 거쳤다고 합니다. 모카와 윤아, 민주 역시 긴 연습생 생활을 했다고 언급되어 있습니다. 반면 원희는 상대적으로 짧은 연습생 생활에도 불구하고 눈에 띄는 성장을 이루었다고 평가되고 있습니다.



Model: claude3_5_sonnet 		 elapsed_time: 4.6 

 Answer: 주어진 문맥에서 연습생 기간에 대한 구체적인 정보는 찾을 수 없습니다. 다만, 윤아와 민주가 "긴 연습생 생활"을 했다는 언급이 있고, 원희는 "비교적 짧은 연습생 생활"을 했다는 내용이 있습니다. 하지만 정확한 기간은 명시되어 있지 않습니다. 따라서 구체적인 연습생 기간에 대해서는 "No Answer found."라고 답변해야 할 것 같습니다.



Model: gpt3_5 		 elapsed_time: 1.4 

 Answer: 윤아와 민주는 연습생 생활을 길게 한 만큼 수준급의 춤 실력을 가지고 있습니다.



Model: gpt4o 		 elapsed_time: 1.1 

 Answer: 윤아와 민주도 긴 연습생 생활을 한 만큼 수준급의 춤 실력을 지녔다.



##### 답변에 사용된 Document 보기

In [100]:
ret.get_relevant_documents(query=question2)

[Document(page_content='일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,, 일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%BD%98%ED%85%90%EC%B8%A0', 'page_hop': 1, 'parent_page_index': '6.2.', 'parent_page_toc_item': '콘텐츠', 'abs_page_toc_item': '활동/콘텐츠//공식채널/10분 논쟁', 'index': '3.5.', 'toc_item': '10분 논쟁', 'ancestor_toc_item': '공식채널'}),
 Document(page_content='공식적인 포지션은 없지만, 모카와 이로하가 주요 댄스 포지션으로 구분된다.  이로하: 만 3세에 전문적인 춤을 추는 영상이 있을 정도로 어릴 적부터 춤 연습을 해왔으며, 파워풀하면서 정확한 춤선이 특징이다.  모카: 과거 댄스 크루원으로 활동했던 만큼 춤 실력이 뛰어나며, 춤선이 깔끔하고 정확하다. 표정연기로도 호평을 받았다.  모카와 이로하 이

### MetaFAISS 벡터 스토어

In [101]:
get_answer(chain_meta, question2)



Model: claude3_haiku 		 elapsed_time: 2.1 

 Answer: 제가 찾아본 내용에는 연습생 생활에 대한 정보가 자세히 나와있지 않습니다. 이 문서에서는 ILLIT의 활동 내역과 일정만 나와있어서 연습생 생활 기간에 대한 정보를 찾을 수 없었습니다. 따라서 "No Answer found."라고 답변드립니다.



Model: claude3_5_sonnet 		 elapsed_time: 3.3 

 Answer: 주어진 문맥에서는 ILLIT 멤버들의 연습생 기간에 대한 정보를 찾을 수 없습니다. 따라서 이 질문에 대해 "No Answer found."라고 답변해야 합니다.



Model: gpt3_5 		 elapsed_time: 13.3 

 Answer: No Answer found.



Model: gpt4o 		 elapsed_time: 0.9 

 Answer: No Answer found.



##### 답변에 사용된 Document 보기

In [102]:
ret_meta.get_relevant_documents(query=question2)

[Document(page_content="일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름, 일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름,", metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%ED%99%9C%EB%8F%99', 'page_hop': 1, 'parent_page_index': '6.', 'parent_page_toc_item': '활동', 'abs_page_toc_item': '활동//데뷔 전', 'index': '2.', 'toc_item': '데뷔 전', 'ancestor_toc_item': ''}),
 Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EA%B3%B5%EC%97%B0%20%EB%B0%8F%20%ED%96%89%EC%82%AC', 'page_hop': 1, 'parent_page_index': '6.3.', 'parent_page_toc_item': '공연 및 행사', 'abs_page_toc_item': '활동/공연 및 행사//쇼케이스'

### 두 벡터 스토어 Ensemble로 사용하기

In [90]:
from langchain.retrievers import EnsembleRetriever

ret_content = vs_meta.as_retriever(vectorStoreType='page_content', search_kwargs={'k': k})
ret_meta = vs_meta.as_retriever(vectorStoreType='metadata', search_kwargs={'k': k})

ret_ensemble = EnsembleRetriever(
    retrievers=[ret_meta, ret_content], weights=[0.5, 0.5]
)

chain_ensemble = get_chain(ret_ensemble, llm)
get_answer(chain_ensemble, question1)



Model: claude3_haiku 		 elapsed_time: 3.1 

 Answer: 본 문서에는 ILLIT의 음반 활동에 대한 정보가 포함되어 있습니다. 이에 따르면 ILLIT은 다음과 같은 음반을 발매했습니다:

- SUPER REAL ME 미니 앨범 (2024년 3월 25일 발매)
- SUPER REAL ME (Sped Up) 리믹스 EP (2024년 4월 1일 발매)
- Magnetic (Remixes) 리믹스 싱글 (2024년 4월 19일 발매)

음반에 대한 구체적인 설명이나 수록곡 정보 등은 제공되지 않습니다. 더 자세한 정보가 필요하시다면 관련 문서를 참고해 주시기 바랍니다.



Model: claude3_5_sonnet 		 elapsed_time: 6.6 

 Answer: ILLIT이 발매한 음반에 대해 다음과 같이 설명할 수 있습니다:

1. 미니 1집 "SUPER REAL ME": 2024년 3월 25일 발매

2. 리믹스 EP "SUPER REAL ME (Sped Up)": 2024년 4월 1일 발매

3. 리믹스 싱글 "Magnetic (Remixes)": 2024년 4월 19일 발매

이렇게 ILLIT은 데뷔 미니앨범과 함께 리믹스 버전의 EP와 싱글도 발매했습니다. 특히 데뷔 앨범인 "SUPER REAL ME"가 가장 먼저 발매되었고, 이후 약 일주일 간격으로 리믹스 버전들이 연이어 발매된 것을 알 수 있습니다.



Model: gpt3_5 		 elapsed_time: 2.6 

 Answer: 해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. ,,,,SUPER REAL ME미니 1집2024. 03. 25.,SUPER REAL ME (Sped Up)리믹스 EP2024. 04. 01.,Magnetic (Remixes)리믹스 싱글2024. 04. 19.



Model: gpt4o 		 elapsed_time: 4.0 

 Answer: 걸그룹 ILLIT이 발

In [106]:
ret_ensemble.get_relevant_documents(query=question1)

[Document(page_content='해당 섹션에 대한 설명이 없거나 하위 문서로 대체됩니다.', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//국내 음반', 'index': '2.', 'toc_item': '국내 음반', 'ancestor_toc_item': ''}),
 Document(page_content='걸그룹 ILLIT이 발매한 음반을 서술한 문서이다. ,,,,SUPER REAL ME미니 1집2024. 03. 25.,SUPER REAL ME (Sped Up)리믹스 EP2024. 04. 01.,Magnetic (Remixes)리믹스 싱글2024. 04. 19.,,', metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%EC%9D%8C%EB%B0%98', 'page_hop': 1, 'parent_page_index': '4.', 'parent_page_toc_item': '음반', 'abs_page_toc_item': '음반//개요', 'index': '1.', 'toc_item': '개요', 'ancestor_toc_item': ''}),
 Document(page_content='ILLIT의 공연 및 행사를 서술한 문서이다.', metadata={'base_page_url': 'https://namu

In [91]:
get_answer(chain_ensemble, question2)



Model: claude3_haiku 		 elapsed_time: 4.2 

 Answer: 알겠습니다. 민주에 대해 다음과 같은 정보를 확인할 수 있습니다:

- 민주는 ILLIT의 멤버로, 그룹의 O형 멤버입니다. 
- 민주는 눈물이 많은 멤버로 알려져 있습니다. 알유넥스트 방송에서 울어버리는 모습이 자주 등장했습니다.
- 민주와 이로하는 '울보즈'라는 조합명으로 불리며, 두 사람 모두 눈물이 많은 멤버라는 공통점이 있습니다.
- 민주는 '슈크림즈'라는 조합명으로도 불렸는데, 이는 민주와 이로하가 둘 다 슈크림 붕어빵을 좋아해서 붙여진 이름입니다. 
- 또한 민주와 모카가 합쳐져 '미모즈'라는 조합명으로 불리기도 합니다.

이상의 정보를 종합해보면, 민주는 ILLIT의 O형 멤버로 눈물이 많은 성격을 가진 것으로 보입니다. 다른 멤버들과 재미있는 조합명도 가지고 있습니다.



Model: claude3_5_sonnet 		 elapsed_time: 10.8 

 Answer: 민주에 대한 정보는 다음과 같습니다:

1. 나이: 데뷔 당시 만 19세입니다.

2. 생일: 5월생입니다.

3. 혈액형: O형입니다.

4. 가족: 오빠가 있습니다.

5. 성격: 눈물이 많은 편으로 알려져 있습니다. '알유넥스트' 4라운드에서 워스트를 받았을 때 울었던 일화가 있습니다.

6. 취향: 슈크림붕어빵을 좋아합니다.

7. 숙소 룸메이트: 원희와 함께 지내고 있습니다.

8. 조합명: 
   - '마희쭈'(마 + 원희 + 민주)
   - '울보즈'(이로하와 함께)
   - '슈크림즈'(이로하와 함께)
   - '미모즈'(모카와 함께)
   - '콩자반즈'(원희, 이로하와 함께) - 민주는 콩자반 헤이터로 알려져 있습니다.

9. 그룹 내 특징: 공식 리더는 없지만, 모든 멤버가 책임감을 가지고 리더처럼 행동한다고 합니다.

이 정보들은 제공된 컨텍스트를 바탕으로 정리되었습니다.



Model: gpt3_5 		 elapsed_time: 2.9 

 Answer

In [104]:
ret_ensemble.get_relevant_documents(query=question2)

[Document(page_content="일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름, 일자,활동,참고자료,비고,2023년,09.02,공식 SNS 개설,,,10.24,I'LL LIKE IT!,,첫 리얼리티,2024년,01.23,ILLIT Ready,,리얼리티,02.26,ILLIT,,ILLIT 로고, 브랜드 필름,", metadata={'base_page_url': 'https://namu.wiki/w/ILLIT', 'parent_page_url': 'https://namu.wiki/w/ILLIT', 'current_page_url': 'https://namu.wiki/w/ILLIT/%ED%99%9C%EB%8F%99', 'page_hop': 1, 'parent_page_index': '6.', 'parent_page_toc_item': '활동', 'abs_page_toc_item': '활동//데뷔 전', 'index': '2.', 'toc_item': '데뷔 전', 'ancestor_toc_item': ''}),
 Document(page_content='일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 간식 몰래 먹는 혈육 | 10분 논쟁 EP.03,, 일자,제목,링크,2024년,04.08,넌 J 난 완전 P 서로를 이해 못 함 영원히 | 10분 논쟁 EP.01,,04.15,비오면 한강 물 💩색 | 여행갈 때 완벽한 날씨 vs 완벽한 교통 | 10분 논쟁 EP.02,,05.08,생각만 해도 열받음 | 방 불 꺼달라고 부르는 혈육 vs 아껴둔 

In [105]:
get_answer(chain_ensemble, "이로하에 대해 설명해줘")



Model: claude3_haiku 		 elapsed_time: 2.7 

 Answer: 이로하에 대해 설명하면 다음과 같습니다:

- ILLIT의 멤버 중 한 명으로, 2024년 3월 25일 데뷔한 다국적 5인조 걸그룹입니다.
- 이로하는 일본인 멤버로, 데뷔 당시 만 16세였습니다.
- 멤버 중 외동이며, 숙소 룸메이트는 윤아와 모카입니다.
- 혈액형은 A형입니다.



Model: claude3_5_sonnet 		 elapsed_time: 8.2 

 Answer: 이로하에 대해 다음과 같은 정보를 찾을 수 있습니다:

1. 이로하는 ILLIT의 멤버입니다.

2. 그녀는 일본인 멤버 중 한 명입니다. (ILLIT은 한국인 3명, 일본인 2명으로 구성됨)

3. 이로하의 생월은 2월입니다.

4. 데뷔 당시(2024년 3월 25일) 이로하의 나이는 만 16세였습니다.

5. 혈액형은 A형입니다.

6. 숙소에서 이로하는 혼자 방을 쓰고 있습니다.

7. 이로하는 외동이입니다.

이 정보들은 주어진 문맥에서 직접적으로 언급된 내용입니다. 이로하에 대한 더 자세한 정보나 특징은 제공된 문맥에서 찾을 수 없습니다.



Model: gpt3_5 		 elapsed_time: 3.0 

 Answer: 이로하는 빌리프랩 소속 걸그룹 ILLIT의 멤버 중 한 명으로, 16세의 나이를 가진 멤버입니다. 이로하는 2월에 태어났으며, 숙소 룸메이트는 민주와 원희입니다. 또한, 이로하는 남동생이 있습니다. 이로하는 과반수 멤버 중 하나이며, ILLIT의 공식 리더가 없다는 사실을 밝힌 멤버 중 하나입니다.



Model: gpt4o 		 elapsed_time: 2.7 

 Answer: 이로하는 ILLIT의 멤버 중 한 명입니다. 2024년 3월 25일에 ILLIT의 멤버로 데뷔하였으며 현재 16세입니다. 혈액형은 A형이고, 생일은 2월입니다. 팀 내에서 외동으로 자랐으며, 숙소에서는 혼자 방을 쓰고 있습니다.



In [109]:
get_answer(chain_ensemble, "이로하와 민주의 케미는 어때?")



Model: claude3_haiku 		 elapsed_time: 2.5 

 Answer: 이로하와 민주는 울보즈 팀 내에서 가장 눈물이 많은 멤버라는 공통점이 있는 것으로 보입니다. 민주는 방송에서 자주 우는 모습이 나오고, 이로하도 눈물이 많은 멤버로 알려져 있습니다. 이들의 조합은 "울보즈"라는 이름으로 불리고 있어, 두 사람 사이에 좋은 케미가 있는 것 같습니다.



Model: claude3_5_sonnet 		 elapsed_time: 7.8 

 Answer: 이로하와 민주의 케미에 대해 다음과 같은 정보를 찾을 수 있습니다:

1. 울보즈: 두 멤버 모두 팀 내에서 가장 눈물이 많은 멤버라는 공통점이 있습니다.

2. 슈크림즈: 원래 둘 다 슈크림붕어빵파라서 붙여진 이름이었지만, 나중에 모카가 슈크림 붕어빵파임이 밝혀졌습니다.

3. 호박아즈: '호'카조노 이로하와 '박'민주, 그리고 노윤'아'를 합한 조합명입니다.

4. 초식즈: 민주는 토끼, 이로하는 아기병아리로 비유되어 초식 동물 컨셉의 조합명입니다.

이처럼 이로하와 민주는 여러 가지 공통점과 특징을 바탕으로 다양한 케미 조합명을 가지고 있어, 팬들에게 인기 있는 조합으로 보입니다.



Model: gpt3_5 		 elapsed_time: 2.6 

 Answer: 이로하와 민주의 케미는 눈물이 많은 멤버로서 서로를 이해하고 지지하는 친절한 모습을 보여주고 있습니다.



Model: gpt4o 		 elapsed_time: 4.7 

 Answer: 이로하와 민주의 케미는 매우 좋습니다. 둘 다 눈물이 많은 멤버라는 공통점이 있어서 "울보즈"라는 조합명으로 불립니다. 이로하는 "알유넥스트" 최종화에서 민주와 원희에게 눈물이 많은 멤버로 뽑혔고, 민주는 "알유넥스트" 4라운드에서 속상함에 눈물을 흘리는 등 방송에서 우는 모습을 자주 보여주었습니다. 이러한 공통점으로 인해 둘은 함께 많은 공감을 나누며 좋은 케미를 보여주고 있습니다.

